## Coding Section for Assignment 2
### Professor Jon Clindaniel, TA Dhruval Bhatt
### Submitted by Junho Choi

This is the Coding Section accompanying the file `junhoc_hw2.pdf`, which contains more descriptive answers. This file contains codes that were used to conduct the assignment as well as related descriptions and annotations.

## Part A. Question 1-(a)

#### A-1. Preparation for the scraping and database operations

In this step, I import the necessary files for the web-scraping job as well as storage to the local `.db` file that I created (called `q1_storage.db`). In case you need to have access to the database I have created, I also upload the `q1_storage.db` file on GitHub as well. I note that there are four tables in the database, `parallel_book_info` and `parallel_books` (for storing the information collected using parallel strategy) and `serial_book_info` and `serial_books` (for that using serial strategy).

In [42]:
## importing necessary packages
import pywren
import numpy as np
import re
import requests
import time
import dataset

from datetime import datetime
from pywren import default_executor as def_exec
from bs4 import BeautifulSoup as BSoup
from urllib.parse import urljoin, urlparse

## setting up the base url and connecting to the DB
base_url = "http://books.toscrape.com"
db = dataset.connect('sqlite:///q1_storage.db')

## checking the tables
print(db.tables)

['parallel_book_info', 'parallel_books', 'serial_book_info', 'serial_books']


#### A-2. Code for scraping all pages
In the parallelization step with `pywren`, I consider using parallelizing by pages on the catalogue. There are 50 pages in total and each page contains 20 items (books). The function below will scrape all the page URLs.

In [52]:
def all_pages(start_url=base_url):
    '''
    This code will collect the list of all pages in the website.

    Input:
    - start_url (str): url to start scraping

    Output:
    - url_lst (list of str): list of page urls (in string format) 

    '''
    url = start_url
    url_list = [start_url]
    start = True
    while True:
        if start:
            print('Collecting the page URLs..')
            start = False
        r = requests.get(url)
        html_soup = BSoup(r.text, 'html.parser')
        next_a = html_soup.select('li.next > a')
        if not next_a or not next_a[0].get('href'):
            break
        url = urljoin(url, next_a[0].get('href'))
        url_list.append(url)

    return url_list

#### A-3. Code for scraping single book information

The below code, given the book-information URL, will scrape the book information on that URL and return the said information in the form of a dictionary. Regarding how to retrieve the said book-information URL used as an input, I will describe this more in **part A-4** below.

In [53]:
def single_book_scraper(book_url):
    '''
    This code will scrape the book information (as well as the time and date
    the scraping happened).
    
    Input:
    - book_url (str): book information url used for scraping book information
    
    Output:
    - book (dict): dictionary containing the book information; keys as
        the column names in the database table, and values as the
        corresponding information
    '''
    r = requests.get(book_url)
    html_soup = BSoup(r.text, 'html.parser')

    ## storing the info in the dictionary as in the example code
    book = dict([])
    book_and_last_seen = dict([])

    ## book ID
    book_id = urlparse(book_url).path.split('/')[2]
    book['book_id'] = book_id
    book_and_last_seen['book_id'] = book_id
    
    ## main part of the product page
    main = html_soup.find(class_='product_main')
    
    ## from main, get title, price, stock
    book['title'] = main.find('h1').get_text(strip=True)
    book['price'] = main.find(class_='price_color').get_text(strip=True)
    book['stock'] = main.find(class_='availability').get_text(strip=True)
    book['rating'] = ' '.join(main.find(class_='star-rating').get(
        'class')).replace('star-rating', '').strip()

    ## getting the image source
    book['image_source'] = html_soup.find(class_='thumbnail').img['src']

    ## description
    desc = html_soup.find(id='product_description')
    book['description'] = ''
    if desc:
        book['description'] = desc.find_next_sibling('p').get_text(
            strip=True)
    
    ## product information table
    info_tbl = html_soup.find(
        text='Product Information').find_next('table')
    
    for row in info_tbl.find_all('tr'):
        header = row.find('th').get_text(strip=True)
        header = re.sub('[^a-zA-Z]', '_', header)
        value = row.find('td').get_text(strip=True)
        book[header] = value

    ## when did the book-info scraping finish approx.ly?
    dt = datetime.now()
    book_and_last_seen['last_seen'] = str(dt)

    return book, book_and_last_seen

#### A-4. Code for scraping books belonging to a single catalogue page

In the below code, the scraper will go through each of the book pages (first by finding the URL for the book then using the above `single_book_scraper` function), get the relevant information in the form of dictionaries, and store (or "upsert") this information in the database table.

In [54]:
def page_scraper(page_url, origin_page=base_url,
                 database=db, books_table_name='serial_books',
                 book_info_table_name='serial_book_info'):
    '''
    This function scrapes all the book information for the books
    on a given page, and upserts it into the specified database's
    table.
    
    Inputs:
    - page_url (str): URL for the catalogue page
    - origin_page (str): the website start page (for creating book URLs)
    - database (dataset.database.Database): database object for connecting
        to the .db file
    - books_table_name (str): database table name to store the book ID and
        last_seen variable
    - book_info_table_name (str) : database table name to store book ID and
        related book information
    
    Output: 
    - None; however, the book information will be
        stored in the database and tables specified.
    '''
    ## checking if it is the first page (the URL is slightly d`ifferent)
    first_page = True
    if 'catalogue' in page_url:
        first_page = False
        
    ## getting the beautifulsoup
    r = requests.get(page_url)
    html_soup = BSoup(r.text, 'html.parser')
    
    ## scraping each book information
    for book in html_soup.select('article.product_pod'):
        ## getting the book URL to send to the single_book_scraper
        book_url = book.find('h3').find('a').get('href')
        if first_page:
            book_url = urljoin(origin_page, book_url)
        else:
            book_url = urljoin(origin_page, 'catalogue/'+book_url)
            
        ## book information scraped, in the form of dictionary
        book_info, books = single_book_scraper(book_url)
        
        ## upserting to the relevant table in the database
        database[book_info_table_name].upsert(book_info, ['book_id'])
        database[books_table_name].upsert(books, ['book_id'])

    return None

#### A-5. Serially running the web scraping process

Below code ensembles the functions above for conducting a serial web-scraping process (including the storage part), for comparing this with the parallelized case using `pywren`.

In [55]:
def serial_scraping(start_url=base_url, database=db,
                    books_table_name='serial_books',
                    book_info_table_name='serial_book_info'):
    ## starting the time
    t0 = time.time()
    
    ## scraping the catalogue page URLs
    page_urls = all_pages(start_url)
    print('Collected all page URLs...')
    
    ## page scraping...
    for i, page in enumerate(page_urls):
        page_num = i+1
        page_scraper(page, start_url, database,
                     books_table_name, book_info_table_name)
        
        ## printing the progress every 10 pages
        if page_num % 10 == 0:
            print("Finished scraping and storing page {}...".format(page_num))
    
    ## finished time output
    t = time.time() - t0
    t = round(t, 4)
    print("Scraping finished; total elapsed time {} seconds.".format(t))
    
    return None

Let us conduct the above-written serial scraping (and storing) code to see how long it takes.

The result shows that the serial process took **550.3733 seconds** approximately.

In [56]:
serial_scraping(base_url, db, 'serial_books', 'serial_book_info')

Collected all page URLs...
Finished scraping and storing page 10...
Finished scraping and storing page 20...
Finished scraping and storing page 30...
Finished scraping and storing page 40...
Finished scraping and storing page 50...
Scraping finished; total elapsed time 550.3733 seconds.


Let us check the number of observations in the tables `serial_book_info` and `serial_books` just in case.

In [59]:
print(db['serial_book_info'].count(), db['serial_books'].count())

1000 1000


Let us also check by printing out some rows of the tables as well.

In [60]:
for i in db['serial_book_info'].find(_limit=3):
    print(i)
    print()

for i in db['serial_books'].find(_limit=5):
    print(i)
    print()

OrderedDict([('book_id', 'a-light-in-the-attic_1000'), ('title', 'A Light in the Attic'), ('price', 'Â£51.77'), ('stock', 'In stock (22 available)'), ('rating', 'Three'), ('image_source', '../../media/cache/fe/72/fe72f0532301ec28892ae79a629a293c.jpg'), ('description', "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh 

#### A-6. Page-level parallelization with pywren

Before the actual parallelization step, I will need to tweak the above `page_scraper` code (into `page_scraper2` described below) so that we can use the code with `pywren`. Instead of directly uploading and passing database as well as the table name, we return the acquired information as a list (so that it can later be inserted / updated / upserted and so forth).

In [61]:
def page_scraper2(page_url, origin_page=base_url):
    '''
    This function scrapes all the book information for the books
    on a given page, and upserts it into the specified database's
    table.
    
    Inputs:
    - page_url (str): URL for the catalogue page
    - origin_page (str): the website start page (for creating book URLs)

    Output: 
    - collect_lst (list of dictionaries): scraped book information,
        later to be used for storage
    '''
    
    ## checking if it is the first page (the URL is slightly different)
    first_page = True
    if 'catalogue' in page_url:
        first_page = False
        
    ## getting the beautifulsoup
    r = requests.get(page_url)
    html_soup = BSoup(r.text, 'html.parser')

    collect_lst_book_info = []
    collect_lst_books = []
    
    ## scraping each book information
    for book in html_soup.select('article.product_pod'):
        ## getting the book URL to send to the single_book_scraper
        book_url = book.find('h3').find('a').get('href')
        if first_page:
            book_url = urljoin(origin_page, book_url)
        else:
            book_url = urljoin(origin_page, 'catalogue/'+book_url)
            
        ## book information scraped, in the form of dictionaries
        book_info, books = single_book_scraper(book_url)
        
        ## appending the information to the lisst to return
        collect_lst_book_info.append(book_info)
        collect_lst_books.append(books)

    return collect_lst_book_info, collect_lst_books

Below code ensembles the functions above for conducting the scraping process, but in a parallelized manner using `pywren`. I note that it still collects the page URLs in a serial manner. Also, note that the entire storage process will take more time if we are "upserting" instead of using `insert_many`. The drawback of the latter case is that it cannot update existing rows, while faster than upserting or inserting row by row, according to the official `dataset` [website](https://dataset.readthedocs.io/en/latest/api.html#table).

In [70]:
def wren_scraping(start_url=base_url, inserting=True,
                  database=db, books_table_name='parallel_books',
                  book_info_table_name='parallel_book_info'):
    '''
    This function utilizes pywren (and therefore, AWS Lambda functions)
    for parallelizing the web-scraping process. Also uploads the data
    to the specified database and table.
    
    Inputs:
    - start_url (str): URL for starting the scraping process
    - inserting (boolean): if True, the process will bunch the scraped
        book information into one list (of dictionaries) and
        dataset-insert_many it; if False, the process will upsert,
        one by one, the information in the list.
    - database (dataset.database.Database): database object for connecting
        to the .db file
    - books_table_name (str): database table name to store the result about
        the book ID and last_seen
    - book_info_table_name (str): database table name to store the result about
        the book ID and related book information

    Output: 
    - None; however, the book information will be
        stored in the database and tables specified.
    '''

    ## starting the time
    t0 = time.time()

    ## scraping the catalogue page URLs
    page_urls = all_pages(start_url)
    print('Collected all page URLs.\n')

    ## lambda function for the parallelized scraping process
    fn = lambda x: page_scraper2(x, start_url)
    
    ## setting up the pywren
    wrenexec = def_exec()
    print('Starting the scraping process...')
    
    ## mapping with pywren; storing the results in a big list of dicts
    futures = wrenexec.map(fn, page_urls)
    results = pywren.get_all_results(futures)

    ## merging into one list of dictionaries
    ## "results" is a list of tuples of lists of dictionaries
    result_lst_book_info = []
    result_lst_books = []
    for result in results:
        result_lst_book_info += list(result[0])
        result_lst_books += list(result[1])
    print('Finished scraping.\n')

    print('Starting the storage process...')
    if inserting:
        length_books = len(result_lst_books)
        length_book_info = len(result_lst_book_info)
        
        database[books_table_name].insert_many(
            result_lst_books, chunk_size=length_books)
        database[book_info_table_name].insert_many(
            result_lst_book_info, chunk_size=length_book_info)
    else:
        for i, book_info in enumerate(result_lst_book_info):
            database[book_info_table_name].upsert(
                book_info, ['book_id'])
            database[books_table_name].upsert(
                result_lst_books[i], ['book_id'])
    print('Finished the storage.\n')

    t = time.time() - t0
    t = round(t, 4)
    print("Process finished; total elapsed time {} seconds.".format(t))
    
    return None

Let us inspect the `insert_many` case (i.e., inserting the rows en masse instead of upserting).

In [71]:
wren_scraping(base_url, True, db,
              'parallel_books', 'parallel_book_info')

Collected all page URLs.

Starting the scraping process...
Finished scraping.

Starting the storage process...
Finished the storage.

Process finished; total elapsed time 34.3902 seconds.


Let us now inspect the `upsert` case.

In [72]:
wren_scraping(base_url, False, db,
              'parallel_books', 'parallel_book_info')

Collected all page URLs.

Starting the scraping process...
Finished scraping.

Starting the storage process...
Finished the storage.

Process finished; total elapsed time 87.7527 seconds.


In summary, we can see that the parallel case with `insert_many` takes **34.3902 seconds** and that with `upsert` takes **87.7527 seconds**.

Finally, let us check the number of observations in the tables `parallel_book_info` and `parallel_books` just in case.

In [78]:
print(db['parallel_book_info'].count(), db['parallel_books'].count())

1000 1000


Let us also check by printing out some rows of the tables as well.

In [75]:
for i in db['parallel_book_info'].find(_limit=3):
    print(i)
    print()

for i in db['parallel_books'].find(_limit=5):
    print(i)
    print()

OrderedDict([('book_id', 'a-light-in-the-attic_1000'), ('title', 'A Light in the Attic'), ('price', 'Â£51.77'), ('stock', 'In stock (22 available)'), ('rating', 'Three'), ('image_source', '../../media/cache/fe/72/fe72f0532301ec28892ae79a629a293c.jpg'), ('description', "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh 

## Part B. Question 2

I note that I have collected the book descriptions in a separate `.csv` file (called `only_descs.csv`) which is also uploaded to the Github repository. This will be used to find the top 10 most frequent words. 

#### B-1. hw2_q2.py for conducting the search

Below is the function to conduct the search for the top-ten most frequent words in the overall book descriptions. I note that the file `hw2_q2.py` is also separately uploaded to the Github repository as well.

Note that in the very last `if` statement, the various `print` statements and timestamps are commented out. This is due to the fact that, if we run the below code in conjunction with EMR, it seems that the said lines cause errors. However, if testing locally, the `print` statements and timestamps will not cause any problems. If run locally using `mrjob` (using the command described in part **B-2**), the said lines can be un-commented.

In [41]:
%%file hw2_q2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import time
import re

WORD_RE = re.compile(r"[\w']+")

class MRTopTenMostUsed(MRJob):
    '''
    Class for conducting the (steps of) mapreduce operations
    to find the top ten count of most frequent words in the
    book descriptions
    
    Input:
    - MRJob: for conducting the mapreduce operation with
        the mrjob package
    
    '''    
    def mapper_get_words(self, _, line):
        '''
        Mapper for preparing the word count
        
        Input:
        - line (str): for creating the word and count pair
        
        Output (yield):
        - tuple (word.lower(), 1): each word (even if redundant)
            is paired with 1, so that the combiner / reducer can
            compute the word counts
        '''
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        '''
        Combiner for the word count
        
        Input:
        - word (str): lowercased word
        - counts (int): should be incoming as 1
        
        Output (yield):
        - tuple (word, sum(counts)): word and the associated
            word count
        '''
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        '''
        Reducer for the word count
        
        Input:
        - word (str): lowercased word
        - counts (int): should be incoming as 1
        
        Output (yield):
        - None: no keys for the next step.
        - tuple (sum(counts), word): word count and the
            associated word
        '''
        yield None, (sum(counts), word)

    def reducer_find_top_ten(self, _, word_count_pairs):
        '''
        Reducer for the finding the top ten cases
        
        Input:
        - word_count_pairs (tuple): word count and the
            associated word
        
        Output (yield):
        - will output the top-nth word and its associated count,
           where n = 1, 2, ..., 10
        '''
        sort_lst = sorted(word_count_pairs, reverse=True)
        for i in range(10):
            yield sort_lst[i][1], sort_lst[i][0]

    def steps(self):
        '''
        Steps function for coordinating the entire
        map reduce process.
        
        Output:
        - rtn_lst: will contain the ending output of the
            entire process.
        '''
        rtn_lst = [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_top_ten)
        ]
        return rtn_lst

if __name__ == '__main__':
    #print("Finding the top-ten most frequent words,"
    #      " from 1st to 10th (with counts)")
    #print()
    #t = time.time()
    MRTopTenMostUsed.run()
    #t1 = round(time.time() - t, 4)
    #print()
    #print("Computation time: {} seconds".format(t1))

Overwriting hw2_q2.py


#### B-2. How to run this from the command line

If we are trying to run this in the command line **locally** (i.e., testing this locally before connecting to AWS EMR), we can type in the below command in the directory where both `hw2_q2_py` and `only_descs.csv` are in:

`python hw2_q2.py < only_descs.csv > q2_local_mrjob.txt`

and the output will be recorded in the text file `q2_local_mrjob.txt` (which is also uploaded to the Github, in the folder `Q2_outputs`).

If we are trying to run this in conjunction with the **AWS EMR**, we can run the following command (in the same directory as mentioned above). Note that `.mrjob.conf` needs to be updated (with the right credentials).

`python hw2_q2.py -r emr < only_descs.csv > q2_emr_mrjob.txt`

and the output will be recorded in the text file `q2_emr_mrjob.txt` (also in the Github folder `Q2_outputs`).

In my application of the codes, running this job locally took approximately **3.3766 seconds**; on the other hand, connecting this with AWS EMR actually took longer, with the computation time of approximately **7 minutes and 40 seconds (460 seconds)**.

#### B-3. Benchmark comparison by not using mrjob

While it is not required by the question, I will compare the computation time with the simple way of finding the top-ten most used words without using `mrjob`. Below block of code is an example (benchmark) code. In this instance, the computation time is approximately **0.1406 seconds** (which is much, much faster).

In [35]:
import pandas as pd
import time
import re

def not_using_mrjob(filename="only_descs.csv"):
    '''
    Performs the top-ten frequent words finding operation
    without using the mrjob package.
    
    Input:
    - filename (str): file name of the csv file containing
        book descriptions.
    
    Output:
    - result (list of tuples): list of top-ten words with their
        respective counts
    - also prints out the computation time
    
    '''
    ## timing starts
    t = time.time()
    
    ## the file doesn't have any header row, so needs some processing
    lines = pd.read_csv("only_descs.csv")
    lines = [lines.columns[0]] + list(lines.iloc[:, 0])
    
    WORD_RE = re.compile(r"[\w']+")
    
    ## cases to be stored in a dictionary
    overall = dict([])
    
    for line in lines:
        for word in WORD_RE.findall(line):
            wordlower = word.lower()
            if overall.get(wordlower) is None:
                overall[wordlower] = 1
            else:
                overall[wordlower] += 1
    
    ## finding the top ten cases
    lst_of_tuples = []
    for i in list(overall.keys()):
        lst_of_tuples.append((overall[i], i))
        
    ## sorting and returning the top ten cases
    sorted_lst = sorted(lst_of_tuples, reverse=True)
    result = sorted_lst[0:10]
    
    t1 = round(time.time()-t, 4)
    print('Computation took {} seconds'.format(t1))
    
    return result

In [36]:
not_using_mrjob()

Computation took 0.1406 seconds


[(13143, 'the'),
 (8693, 'and'),
 (7878, 'of'),
 (7085, 'a'),
 (6089, 'to'),
 (4333, 'in'),
 (3133, 'is'),
 (2510, 'her'),
 (2105, 'that'),
 (1999, 'with')]

## Part C. Question 3-(a)

In this part, most of the codes will actually be launched from the Jupyter Notebook itself (unless noted otherwise). I also note that I heavily borrow from Professor Clindaniel's Lab 5 Part II codes.

#### C-1. Setting up Kinesis and EC2 clients

I borrow from Professor Clindaniel's code to set up the Kinesis and EC2 clients.

In [100]:
import boto3

## setting up clients for kinesis and EC2 instance
session = boto3.Session()
kinesis = session.client('kinesis')
ec2 = session.resource('ec2')
ec2_client = session.client('ec2')

#### C-2. Creating the Kinesis stream

Since we only have one shard in our AWS Educate accounts, I made sure that all other Kinesis streams are terminated and deleted before running the below code.

In [101]:
## creating the stream
stream_name = 'q3_stock_stream'
response = kinesis.create_stream(StreamName = stream_name,
                                 ShardCount = 1)

## waiting until the stream creation process is actually finished
waiter = kinesis.get_waiter('stream_exists')
waiter.wait(StreamName=stream_name)

#### C-3. Creating the EC2 instances

I now create the EC2 instances. We will need two of them, one for the `producer` and the other for the `consumer`. I note that my permission key is also named `MACS_30123`, but for the security group (and its ID) I use different values (written below).

In [102]:
# my security-group-related values
security_id, security_group = 'sg-0726a1a64e8f26e6d', 'junhoc-lab5-q3'

# creating the instances
instances = ec2.create_instances(
    ImageId='ami-0915e09cc7ceee3ab', MinCount=1, MaxCount=2,
    InstanceType='t2.micro', KeyName='MACS_30123',
    SecurityGroupIds=[security_id], SecurityGroups=[security_group],
    IamInstanceProfile={'Name': 'EMR_EC2_DefaultRole'}, )

# making sure that the EC2 instances are created
waiter = ec2_client.get_waiter('instance_running')
waiter.wait(InstanceIds=[instance.id for instance in instances])

#### C-4. Script for the Producer instance

Following the Lab 5 notebook and the code provided in the Assignment file, I set up the script for the Producer instance as follows.

In [103]:
%%file q3_producer.py

import boto3
import random
import datetime
import json

## setup for the Kinesis client and the stream name
stream_name = 'q3_stock_stream'
kinesis = boto3.client('kinesis', region_name='us-east-1')

## code for generating stock prices (in USD terms)
def getReferrer():
    data = {}
    now = datetime.datetime.now()
    str_now = now.isoformat()
    data['EVENT_TIME'] = str_now
    data['TICKER'] = 'AAPL'
    price = random.random() * 100
    data['PRICE'] = round(price, 2)
    return data

## code for putting the generated stock price into Kinesis
while True:
    kinesis.put_record(StreamName=stream_name,
                       Data=json.dumps(getReferrer()),
                       PartitionKey="partitionkey")

Overwriting q3_producer.py


#### C-5. Script for the Consumer instance

Following the Lab 5 notebook, I set up the script for the Consumer instance as follows.

In [104]:
%%file q3_consumer.py

import boto3
import time
import json
from datetime import datetime as dt

## setup for the Kinesis client and the stream name
stream_name = 'q3_stock_stream'
kinesis = boto3.client('kinesis', region_name='us-east-1')

## shard iterator
shard_it = kinesis.get_shard_iterator(
    StreamName=stream_name, ShardId='shardId-000000000000',
    ShardIteratorType='LATEST')["ShardIterator"]

while True:
    ## Getting the stock price output
    out = kinesis.get_records(ShardIterator=shard_it, Limit=1)
    
    ## all the records
    for o in out['Records']:
        stock_data = json.loads(o['Data'])

    ## setting the latest values to be printed
    time_info = stock_data['EVENT_TIME']
    ticker_info = stock_data['TICKER']
    price_info = stock_data['PRICE']
        
    print("Date and time: {}".format(time_info))
    print("Current price for {}: {}".format(ticker_info, price_info))
    print("\n")
        
    shard_it = out['NextShardIterator']
    time.sleep(0.2)

Overwriting q3_consumer.py


#### C-6. Grabbing the DNS names of the instances (for convenience)

I follow Professor Clindaniel's code for grabbing the DNS names.

In [105]:
instance_dns = [
    instance.public_dns_name for instance in ec2.instances.all() 
    if instance.state['Name'] == 'running']

code = ['q3_producer.py', 'q3_consumer.py']

#### C-7. Finalizing the consumer and producer instances

I also follow Professor Clindaniel's code for finalizing the establishment of consumer and producer instances. Note that there is a very minor change about using `enumerate` in the for loop and the `sudo pip install` statement for producer not requiring the `testdata` module. By running the below codes, the producer- and consumer-instances setup is complete.

In [106]:
import paramiko
import time
from scp import SCPClient

## where my pem file is at
my_pem_dir = 'C:/Users/Owner/MACS_30123.pem'

In [107]:
## setting up the SSH
ssh_producer, ssh_consumer = paramiko.SSHClient(), paramiko.SSHClient()

# Initialization of SSH tunnels takes a bit of time; otherwise get connection error on first attempt
time.sleep(5)

# Install boto3 on each EC2 instance and Copy our producer/consumer code onto producer/consumer EC2 instances
stdin, stdout, stderr = [[None, None] for i in range(3)]
for instance, ssh in enumerate([ssh_producer, ssh_consumer]):
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(instance_dns[instance], username = 'ec2-user',
                key_filename=my_pem_dir)
    
    with SCPClient(ssh.get_transport()) as scp:
        scp.put(code[instance])

    ## producer case
    if instance == 0:
        stdin[instance], stdout[instance], stderr[instance] = \
            ssh.exec_command("sudo pip install boto3")
    ## consumer case
    else:
        stdin[instance], stdout[instance], stderr[instance] = \
            ssh.exec_command("sudo pip install boto3")

# Block until Producer has installed boto3; then start running Producer script
producer_exit_status = stdout[0].channel.recv_exit_status() 
if producer_exit_status == 0:
    ssh_producer.exec_command("python %s" % code[0])
    print("Producer Instance is Running producer.py\n.........................................")
else:
    print("Error", producer_exit_status)

# Close ssh and show connection instructions for manual access to Consumer Instance
ssh_consumer.close(); ssh_producer.close()

print("Connect to Consumer Instance by running: ssh -i \"MACS_30123.pem\" ec2-user@%s" % instance_dns[1])

Producer Instance is Running producer.py
.........................................
Connect to Consumer Instance by running: ssh -i "MACS_30123.pem" ec2-user@ec2-54-210-250-53.compute-1.amazonaws.com


## Part D. Question 3-(b)

In this part, most of the codes will actually be launched from the Jupyter Notebook itself (unless noted otherwise). I also note that I heavily borrow from the Datacamp Assignment 1's part on using `SNS`.

#### D-1. Creating the topic

Following the instructions, I first create the topic `Price_Alert`.

In [108]:
import boto3

# initialize boto3 client for sns
session2 = boto3.Session()
sns = session2.client('sns')

# create the Price_Alert topic
pa = 'Price_Alert'
response = sns.create_topic(Name=pa)
alert_arn = response['TopicArn']

# checking the alert_arn
print(alert_arn)

arn:aws:sns:us-east-1:608865353210:Price_Alert


#### D-2. Subscribing my UChicago email

I now subscribe my UChicago email (being `junhoc@uchicago.edu`) to the `Price_Alert` topic. I note that after having done so, I logged into my email account and confirmed my subscription.

In [109]:
## I use the previously-created alert_arn
subscribe_email = sns.subscribe(
    TopicArn=alert_arn, Protocol='email',
    Endpoint='junhoc@uchicago.edu')

#### D-3. Initializing the Kinesis Stream

Let us initialize the Kinesis Stream as in the Datacamp assignment.

In [110]:
## setting up the kinesis stream (consumer-side)
stream_name = 'q3_stock_stream'
kinesis = boto3.client('kinesis', region_name='us-east-1')

shard_it = kinesis.get_shard_iterator(
    StreamName=stream_name, ShardId='shardId-000000000000',
    ShardIteratorType='LATEST')["ShardIterator"]

#### D-5. Getting the stock information and sending the price alert

The below code accesses the Kinesis stream (from the consumer side). As soon as the price is below \$3.00, it will send the alert via email, terminate or delete the related EC2 instances, SNS topic, and Kinesis Stream. I have posted the screenshot of the price alert email I received in the `junhoc_hw2.pdf` file.

In [111]:
import pandas as pd
import time

while True:
    ## Getting the stock price output
    out = kinesis.get_records(ShardIterator=shard_it, Limit=1)
    
    ## the most recent record
    for o in out['Records']:
        stock_data = json.loads(o['Data'])

    ## setting the latest values to be printed
    time_info = stock_data['EVENT_TIME']
    ticker_info = stock_data['TICKER']
    price_info = stock_data['PRICE']
        
    shard_it = out['NextShardIterator']
    time.sleep(0.2)

    ## see part C-1. for "ec2_client" (EC2 client)
    ## see part C-3. for "instances" (storing EC2 instances)
    if price_info < 3.0:
        ## printing the stock information, just in case
        print("Date and time: {}".format(time_info))
        print("Current price for {}: {}".format(ticker_info, price_info))
        print("\n")
        
        ## sending the price alert
        msg = "Price for {} is below $3.00; currently at ${}".format(
            ticker_info, price_info)
        sbj = "Urgent price alert for {}".format(ticker_info)
        sns.publish(TopicArn=alert_arn, Subject=sbj, Message=msg)
            
        ## terminating the EC2 instances
        ec2_client.terminate_instances(
            InstanceIds=[instance.id for instance in instances])
        print("Waiting for the EC2 instances to be terminated...")
        
        ## waiting for the EC2 instances to terminate
        waiter = ec2_client.get_waiter('instance_terminated')
        waiter.wait(InstanceIds=[instance.id for instance in instances])
        print("EC2 instances successfully terminated.")
        print("\n")
        
        ## deleting the SNS topic
        sns.delete_topic(TopicArn=alert_arn)
        print("Waiting for the topic to be deleted...")
        time.sleep(3)
        
        ## making sure that the SNS topic for Price Alert no longer exists
        ## I checked that there are no "waiters" for SNS
        lst_of_arns = list(pd.DataFrame(
            sns.list_topics()['Topics'])['TopicArn'])
        if alert_arn not in lst_of_arns:
            print("{} topic has been successfully deleted.".format(pa))
        else:
            print("Error in deleting the topic {}".format(pa))
        print("\n")
        
        ## Deleting the Kinesis Stream
        ## "stream_name" was defined in Part C-2.
        try:
            response = kinesis.delete_stream(StreamName=stream_name)
        except kinesis.exceptions.ResourceNotFoundException:
            pass
        
        ## making sure the stream is deleted
        print("Waiting for the stream to be deleted...")
        waiter = kinesis.get_waiter('stream_not_exists')
        waiter.wait(StreamName='test_stream')
        print("Kinesis stream successfully deleted.")
        
        break

Date and time: 2020-05-19T22:12:28.213621
Current price for AAPL: 1.77


Waiting for the EC2 instances to be terminated...
EC2 instances successfully terminated.


Waiting for the topic to be deleted...
Price_Alert topic has been successfully deleted.


Waiting for the stream to be deleted...
Kinesis stream successfully deleted.
